In [ ]:
def fetch_yahoo_info_dataframe(
    tickers: Iterable[str],
    sleep_seconds: float = 1,
    max_retries: int = 3,
    retry_backoff: float = 2.0,
    jitter: float = 0.3,
    verbose: bool = True,
) -> Tuple[pd.DataFrame, List[str]]:
    """
    Fetch yf.Ticker(ticker).info for many tickers in a robust and rate-limit-friendly way.

    Parameters
    ----------
    tickers : Iterable[str]
        Iterable of Yahoo Finance tickers
    sleep_seconds : float
        Base sleep time between requests
    max_retries : int
        Number of retries per ticker
    retry_backoff : float
        Exponential backoff factor for retries
    jitter : float
        Random jitter added to sleep to avoid request patterns
    verbose : bool
        Print progress information

    Returns
    -------
    df_info : pd.DataFrame
        DataFrame containing all collected info dicts
    failed_tickers : list[str]
        List of tickers for which all retries failed
    """

    records = []
    failed_tickers = []

    tickers = list(tickers)
    total = len(tickers)

    for i, ticker in enumerate(tickers, start=1):
        if verbose:
            print(f"[{i}/{total}] Fetching {ticker}")

        success = False
        last_exception = None

        for attempt in range(1, max_retries + 1):
            try:
                info = yf.Ticker(ticker).info

                # Yahoo sometimes returns empty dicts
                if not info or not isinstance(info, dict):
                    raise ValueError("Empty or invalid info dictionary")

                # add ticker explicitly (important for later joins)
                info["yf_ticker"] = ticker

                records.append(info)
                success = True
                break

            except Exception as e:
                last_exception = e
                if verbose:
                    print(
                        f"  Attempt {attempt}/{max_retries} failed for {ticker}: {e}"
                    )

                sleep_time = sleep_seconds * (retry_backoff ** (attempt - 1))
                sleep_time += random.uniform(0, jitter)
                time.sleep(sleep_time)

        if not success:
            failed_tickers.append(ticker)
            if verbose:
                print(f"  ❌ Failed permanently: {ticker} ({last_exception})")

        # base sleep between tickers
        time.sleep(sleep_seconds + random.uniform(0, jitter))

    df_info = pd.DataFrame(records)

    return df_info, failed_tickers
